In [1]:
import os
import dotenv
import numpy as np
import pandas as pd
from ieeg.auth import Session  # >_ cd ieegy --> pip install -e .

In [2]:
dotenv.load_dotenv()

session = Session(os.getenv('IEEG_USERNAME'), os.getenv('IEEG_PASSWORD'))
dataset = session.open_dataset(os.getenv('IEEG_DATASET'))

In [3]:
# Get the channel labels
channel_labels = dataset.get_channel_labels()
num_channels = len(channel_labels)

# Get the temporal details of the first channel
timeseries = dataset.get_time_series_details(channel_labels[0])
fs = int(timeseries.sample_rate)  # Hz

print(f"Sampling rate: {fs} Hz")

Sampling rate: 400 Hz


In [4]:
# Data to pull
total_mins = 60
chunk_mins = 10
chunk_usec = chunk_mins * 60 * 1e6  # Convert to microseconds

# Initialize empty list to store chunks
dfs = []

# Loop through chunks
for i in range(total_mins // chunk_mins):
    start_usec = i * chunk_usec
    print(f"Collecting chunk {i+1}/{int(total_mins/chunk_mins)}: Minutes {i*chunk_mins} to {(i+1)*chunk_mins}")
    
    # Get data chunk
    chunk_df = dataset.get_dataframe(start_usec, chunk_usec, np.arange(num_channels))
    dfs.append(chunk_df)

# Concatenate all chunks
full_df = pd.concat(dfs, ignore_index=True)
print(f"Dataset shape: {full_df.shape}")

Dataset shape: (1440000, 16)


In [5]:
# Drop NaN rows
full_df_no_nan = full_df.dropna()
print(f"\nFinal dataset shape: {full_df_no_nan.shape}")


Final dataset shape: (28850, 16)


In [7]:
# Save dataset to CSV
data_filename = 'data.csv'
data_filepath = os.path.join('..', 'data', 'input', data_filename)

full_df_no_nan.to_csv(data_filepath)
print(f"Data saved to {data_filepath}")

Data saved to ../data/input/data.csv
